In [1]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
from datetime import datetime
from dateutil import tz
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker


import pynwb
from pynwb import register_class
from pynwb.form.utils import docval, getargs, popargs, call_docval_func

import nspike_helpers as ns 
import query_helpers as qu

mdates.rcParams.update({'date.autoformatter.microsecond': '%H:%M:%S.%f'})

/Users/ericmiller/anaconda/envs/pynwb/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Overview
* **FL_BehavioralEpochs** - A DynamicTable where each row contains data from an epoch, possibly including:

    * **name** - Name of the epoch
    * **position** - Position (multicontainer of SpatialSeries) for animal position during the epoch, possibly including:
        * {(x1, y1)...} - 2D position vs time, with respect to a world geometry (e.g. a room or a camera frame)
        * {d1, ...} - 1D distance vs time, linked to a 1D linearized track geometry    
    * **head_dir** - CompassDirection (multicontainer of SpatialSeries) for animal head direction during the epoch
    * **speed** - BehavioralTimeSeries (multicontainter for TimeSeries) for animal speed during the epoch
    * **FL_BehavioralEvents** - DynamicTable of possibly-overlapping behavioral events (e.g. cue light on, reward delivery on)
    * **FL_BehavioralGraphPath** - TimeSeries or non-overlapping TimeIntervals of animal's path through the FL_BehavioralGraph in this epoch
    * **FL_BehavioralTask** - Container for data describing the task/apparatus for the epoch (a task instance should be reusable), possibly including:
        * **FL_BehavioralGraph** - graph representing the structure of discrete, non-overlapping states of the behavior (e.g. on segment 3, not at well)
        * **FL_BehavioralWorld** - collection of geometries describing the behavioral world (either with respect to a room geometry or with respect to a camera frame) possibly including:
            * 1D linearized representation of the track
            * 2D geometries representing track borders
            * 2D or 3D geometries representing the experimental room
            * n-D geoemtries for objects, cues, or other elements of the world (e.g. locations of wells, cues, levers)
     
    

### Spec for a Behavioral Task group
An object for representing a behavioral task structure. A given instance of FL_BehavioralTask should be reusable for different epochs of one animal or between animals, assuming that a task is ever used more than once. It could include:

* **FL_BehavioralWorld** - geometries describing the spatial and operant world of this task (e.g. 1D and 2D track geometry, the room, notable objects or cues). These world geometries can then be linked to animal position (e.g. linking 2D w-track position data to an instance of the W-track in the same reference frame)
* **FL_BehavioralGraph** - discrete, non-overlapping states of animal behavior (e.g. open-field no well, open-field at well, arm 4 no well, arm 4 at well, ...)
* Attributes such as human-readable descriptions of the task and apparatus.

In [2]:
world_spec = pynwb.spec.NWBGroupSpec(name='world',
                                     doc='Collection of geometries (1D, 2D, or 3D) describing the behavioral environment',
                                     neurodata_type_inc='NWBContainer',
                                     neurodata_type_def='FL_BehavioralWorld')

graph_spec = pynwb.spec.NWBGroupSpec(name='graph',
                                   doc='Graph describing non-overlapping behavioral states',
                                   neurodata_type_inc='NWBContainer',
                                   neurodata_type_def='FL_BehavioralGraph')

behavioral_task_spec = pynwb.spec.NWBGroupSpec(name='FL_BehavioralTask',
                                               doc='Container of objects for describing a behavioral task',
                                               groups=[world_spec, graph_spec],
                                               neurodata_type_inc='NWBContainer',
                                               neurodata_type_def='FL_BehavioralTask')

### Spec for a FL_BehavioralEvents DynamicTable
DynamicTable of possibly-overlapping instances of experimental/behavioral events (e.g. reward delivered, cue light on).

This spec is currently just a template.


In [3]:
behavioral_events_spec = pynwb.spec.NWBGroupSpec(name='events',
                                      doc='DynamicTable of describing possibly-overlapping behavioral/experimental events',
                                      neurodata_type_inc='DynamicTable',
                                      neurodata_type_def='FL_BehavioralEvents')

### Spec for FL_BehavioralGraphPath
BehavioralTimeSeries describing the animal's non-overlapping path through a FL_BehavioralGraph in this epoch. This is the path through a graph for a given epoch, not the structure of the graph itself. The structure of the graph is described in the FL_BehavioralGraph, usually found within an FL_BehavioralTask object. 

This spec is currently just a template.

In [4]:
behavioral_graph_path_spec = pynwb.spec.NWBGroupSpec(name='graph_path',
                                        doc='Non-overlapping path through a FL_BehavioralGraph',
                                      neurodata_type_inc='BehavioralTimeSeries',
                                      neurodata_type_def='FL_BehavioralGraphPath')

### Spec for an Epochs DynamicTable
Each epoch (row) of the table contains:
* Name (text)
* Task (FL_BehavioralTask)
* Position (pynwb.behavior.Position)
* Speed (pynwb.behavior.BehavioralTimeSeries)
* Head direction (pynwb.behavior.CompassDirection)

In [5]:
epoch_datasets = [pynwb.spec.NWBDatasetSpec(name='epoch_name',
                                            doc='Name for this epoch',
                                            dtype='text',
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                  pynwb.spec.NWBDatasetSpec(name='position', 
                                            doc='Animal spatial position during an epoch', 
                                            dtype=pynwb.spec.NWBRefSpec('Position', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                  pynwb.spec.NWBDatasetSpec(name='speed', 
                                            doc='Animal speed during an epoch', 
                                            dtype=pynwb.spec.NWBRefSpec('BehavioralTimeSeries', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                  pynwb.spec.NWBDatasetSpec(name='head_dir', 
                                            doc='Animal head direction during an epoch',
                                            dtype=pynwb.spec.NWBRefSpec('CompassDirection', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                 pynwb.spec.NWBDatasetSpec(name='task',
                                            doc='Task used in this epoch',
                                            dtype=pynwb.spec.NWBRefSpec('FL_BehavioralTask', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                 pynwb.spec.NWBDatasetSpec(name='events',
                                            doc='Events occurring in this epoch',
                                            dtype=pynwb.spec.NWBRefSpec('FL_BehavioralEvents', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                 pynwb.spec.NWBDatasetSpec(name='graph_path',
                                            doc='Path through a BehavioralGraph in this epoch',
                                            dtype=pynwb.spec.NWBRefSpec('FL_BehavioralGraphPath', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData')]

behavioral_epochs_spec = pynwb.spec.NWBGroupSpec(name='FL_BehavioralEpochs',
                                                 doc='DynamicTable for describing behavioral epochs',
                                                 neurodata_type_inc='DynamicTable', 
                                                 neurodata_type_def='FL_BehavioralEpochs',
                                                 datasets=epoch_datasets,
                                                 quantity="?")

### Build a namespace and export to YAML

In [6]:
ns_path = "franklab.namespace.yaml"
ext_path = "franklab.extensions.yaml"

ns_builder = pynwb.spec.NWBNamespaceBuilder('Extension for use in the Frank Lab', "franklab")

ns_builder.add_spec(ext_path, behavioral_task_spec)
ns_builder.add_spec(ext_path, behavioral_events_spec)
ns_builder.add_spec(ext_path, behavioral_graph_path_spec)
ns_builder.add_spec(ext_path, behavioral_epochs_spec)

ns_builder.export(ns_path)

### Load our namespace

In [7]:
pynwb.load_namespaces(ns_path)

('franklab',)

### Auto-generate a Python class for our extensions

#### Currently does not work...
Auto-generation seems to not work with DynamicTables yet.
It correctly creates a class inheriting from DynamicTable, but it does not implement the columns
in the "columns" class variable or the docstring in "add_row()". Recall that in the Units DynamicTable,
a new method "add_unit()" was created with a docstring for the correct columns (e.g. spiketimes, obs_intervals).
A similar "add_epoch()" method is not auto-generated.

In [8]:
from pynwb import get_class
# FL_BehavioralEpochs = get_class('FL_BehavioralEpochs', 'franklab')  # This is currently not working
FL_BehavioralWorld = get_class('FL_BehavioralWorld', 'franklab')
FL_BehavioralGraph = get_class('FL_BehavioralGraph', 'franklab')
FL_BehavioralEvents = get_class('FL_BehavioralEvents', 'franklab')    # This one will also likely not work once we actually define the table columns.
FL_BehavioralGraphPath = get_class('FL_BehavioralGraphPath', 'franklab')
# FL_BehavioralTask = get_class('FL_BehavioralTask', 'franklab')      # This is currently not working

### Manually create a class for FL_BehavioralEpochs

In [10]:
@register_class('FL_BehavioralEpochs', 'franklab')
class FL_BehavioralEpochs(pynwb.core.DynamicTable):
    """
    Data for behavioral epochs.
    """

    # Setting index to True throws an error if I try to store non-iterable types, such as SpatialSeries
    __columns__ = (
        {'name': 'epoch_name', 'description': 'Name for this epoch', 'index': False},
        {'name': 'position', 'description': 'Animal spatial position during an epoch', 'index': False},
        {'name': 'speed', 'description': 'Animal speed during an epoch', 'index': False},
        {'name': 'head_dir', 'description': 'Animal head direction during an epoch', 'index': False},
        {'name': 'task', 'description': 'Behavioral task for this epoch', 'index': False},
        {'name': 'events', 'description': 'Possibly-overlapping behavioral events in this epoch', 'index': False},
        {'name': 'graph_path', 'description': 'Non-overlapping graph path for this epoch', 'index': False}
    )

    @docval({'name': 'name', 'type': str, 'doc': 'Name of this FL_BehavioralEpochs table', 'default': 'FL_BehavioralEpochs'},
            {'name': 'id', 'type': ('array_data', pynwb.core.ElementIdentifiers),
             'doc': 'the identifiers for the epochs stored in this interface', 'default': None},
            {'name': 'columns', 'type': (tuple, list), 'doc': 'the columns in this table', 'default': None},
            {'name': 'colnames', 'type': 'array_data', 'doc': 'the names of the columns in this table',
             'default': None},
            {'name': 'description', 'type': str, 'doc': 'a description of what is in this FL_BehavioralEpochs table', 'default': None})
    def __init__(self, **kwargs):
        if kwargs.get('description', None) is None:
            kwargs['description'] = ""
        call_docval_func(super(FL_BehavioralEpochs, self).__init__, kwargs)

    @docval({'name': 'epoch_name', 'type': str, 'doc': 'Name for this epoch', 'default': None},
            {'name': 'position', 'type': pynwb.behavior.Position, 'doc': 'Animal spatial position during an epoch', 'default': None},
            {'name': 'speed', 'type': pynwb.behavior.BehavioralTimeSeries, 'doc': 'Animal speed during an epoch', 'default': None},
            {'name': 'head_dir', 'type': pynwb.behavior.CompassDirection, 'doc': 'Animal head direction during an epoch', 'default': None},
            {'name': 'task', 'type': 'FL_BehavioralTask', 'doc': 'Behavioral task for this epoch', 'default': None},
            {'name': 'events', 'type': 'FL_BehavioralEvents', 'doc': 'Possibly-overlapping behavioral events in this epoch', 'default': None},
            {'name': 'graph_path', 'type': 'FL_BehavioralGraphPath', 'doc': 'Non-overlapping graph path for this epoch', 'default': None},
            {'name': 'id', 'type': int, 'default': None, 'help': 'the id for each unit'},
            allow_extra=True)
    def add_epoch(self, **kwargs):
        """
        Add an epoch to this table
        """
        super(FL_BehavioralEpochs, self).add_row(**kwargs)


### Manually create a class for FL_BehavioralTask

In [11]:
@register_class('FL_BehavioralTask', 'franklab')
class FL_BehavioralTask(pynwb.core.NWBContainer):
    __nwbfields__ = ('name', 'world', 'graph')

    @docval({'name': 'name', 'type': str, 'doc': 'who names a potato?', 'default': 'FL_BehavioralTask'},
            {'name': 'world', 'type': 'FL_BehavioralWorld', 'doc': 'Collection of geometries (1D, 2D, or 3D) describing the behavioral environment', 'default': None},
            {'name': 'graph', 'type': 'FL_BehavioralGraph', 'doc': 'Graph describing non-overlapping behavioral states', 'default': None})
    def __init__(self, **kwargs):
        super(FL_BehavioralTask, self).__init__(name=kwargs['name'])
        self.world = kwargs['world']
        self.graph = kwargs['graph']

### Extract behavioral data

In [12]:
# Session-specific params
data_dir = os.path.expanduser('~/Data/FrankData/kkay/Bon')
anim = 'Bon' 
prefix = anim.lower()
day = 3 # below we'll code date as 2006-Jan-'Day'

# Calculate the POSIX timestamp when Nspike clock = 0 (seconds)
dataset_zero_time = datetime(2006, 1, day, 12, 0, 0, tzinfo=tz.gettz('US/Pacific'))
NSpike_posixtime_offset = dataset_zero_time.timestamp()

# NOTE that day_inds is 0 based
time_list = {}
nwb_epoch = {}
pos_files = ns.get_files_by_day(data_dir, prefix, 'pos')
task_files = ns.get_files_by_day(data_dir, prefix, 'task')

mat = ns.loadmat_ff(task_files[day], 'task')
task_struct = mat[day]

mat = ns.loadmat_ff(pos_files[day], 'pos')
pos_struct = mat[day]

# create position, direction and speed
position_list = []
direction_list = []
speed_list = []
time_list = []

# Assume field order: (time,x,y,dir,vel)
(time_idx, x_idx, y_idx, dir_idx, vel_idx) = range(5)

for epoch_num, pos_epoch in pos_struct.items():
    # convert times to POSIX time
    timestamps = pos_epoch['data'][:,time_idx] + NSpike_posixtime_offset
    # collect times of epoch start and end
    time_list.append([timestamps[0], timestamps[-1]])
    m_per_pixel = pos_epoch['cmperpixel'][0,0]/100 # NWB wants meters per pixel

    # These are multicontainers, so we can include multiple Spatial series for each epoch
    # For example, 2D (x, y) position with respect to a camera frame or room geometry, and 1D linearized position
    positions = pynwb.behavior.Position(name='Position for Epoch ' + str(epoch_num))
    directions = pynwb.behavior.CompassDirection(name='Head direction for Epoch ' + str(epoch_num))
    speeds = pynwb.behavior.BehavioralTimeSeries(name='Speed for Epoch ' + str(epoch_num))
    
    positions.create_spatial_series(name='2D position d%d e%d' % (day, epoch_num), 
                                      timestamps = timestamps,
                                      data=pos_epoch['data'][:, (x_idx, y_idx)] * m_per_pixel,
                                      reference_frame='corner of video frame',
                                      #conversion=m_per_pixel,
                                      #unit='m'
                                      ) # *after* conversion
    position_list.append(positions)
    
    directions.create_spatial_series(name='Head direction d%d e%d'% (day, epoch_num), 
                              timestamps=timestamps,
                              data=pos_epoch['data'][:, dir_idx],
                              reference_frame='0=facing top of video frame (?), positive clockwise (?)',
                              #unit='radians'
                              )
    direction_list.append(directions)
    
    speeds.create_timeseries(name='Speed d%d e%d' % (day, epoch_num),
                             timestamps=timestamps,
                             data=pos_epoch['data'][:, vel_idx] * m_per_pixel,
                             unit='m/s',
                             #conversion=m_per_pixel,
                             description='smoothed movement speed estimate')
    speed_list.append(speeds)
    
time_list = np.asarray(time_list)

### Implement an Epochs table using all of our extensions

In [17]:
# Define two tasks for this data
# --- W track -----
w_track_world = FL_BehavioralWorld(name='W-track World')
w_track_graph = FL_BehavioralGraph(name='W-track Graph')
w_track_task = FL_BehavioralTask(name='W-track Task', world=w_track_world, graph=w_track_graph)
# --- Sleep box -----
sleep_box_world = FL_BehavioralWorld(name='Sleep Box World')
sleep_box_graph = FL_BehavioralGraph(name='Sleep Box Graph')
sleep_box_task = FL_BehavioralTask(name='Sleep Box Task', world=sleep_box_world, graph=sleep_box_graph)

# Epochs table
num_epochs = len(pos_struct)
epochs = FL_BehavioralEpochs(name='Epochs for Bon day 3')
for epoch_idx in range(num_epochs):   
    # Alternate between W track and sleep epochs <--- This is probably not matching the data.
    if epoch_idx % 2 == 0:
        task = w_track_task
    else:
        task = sleep_box_task
    # Get the events table for this epoch
    events = FL_BehavioralEvents(name='Events epoch ' + str(epoch_idx), 
                                 description='Events for epoch ' + str(epoch_idx))
    # Get the graph path for this epoch
    graph_path = FL_BehavioralGraphPath(name='Graph path, epoch ' + str(epoch_idx))
    # Add a row to the epochs table for this epoch
    epochs.add_epoch(epoch_name='Epoch ' + str(epoch_idx), 
                     position=position_list[epoch_idx], 
                     speed=speed_list[epoch_idx], 
                     head_dir=direction_list[epoch_idx],
                     task=task,
                     events=events,
                     graph_path=graph_path)
    

### Inspect our Epochs table

In [18]:
print("An Epochs DynamicTable\n--------------")
print(epochs)
print('\n')

print("Position in epoch 2\n------------")
print(epochs[1, 'position'])
print('\n')

print("Speed in epoch 4\n------------")
print(epochs[3, 'speed'])
print('\n')

print("Head direction in epoch 1\n------------")
print(epochs[0, 'head_dir'])
print('\n')

print("Events in epoch 3\n------------")
print(epochs[2, 'events'])
print('\n')

print("Graph path in epoch 5\n------------")
print(epochs[4, 'graph_path'])
print('\n')

print("Task for the final epoch\n----------")
print(">>> Currently the sub-groups don't contain any real data.")
print(">>> For some reason inspecting task shows the sub-groups as being None, when in fact they are not...")
print(epochs[-1, 'task'])
print(">>> But we can access them individually...")
print(epochs[-1, 'task'].world)
print(epochs[-1, 'task'].graph)
print('\n')

An Epochs DynamicTable
--------------

Epochs for Bon day 3 <class '__main__.FL_BehavioralEpochs'>
Fields:
  colnames: ('epoch_name', 'position', 'speed', 'head_dir', 'task', 'events', 'graph_path')
  columns: (
epoch_name <class 'pynwb.core.VectorData'>
Fields:
  description: Name for this epoch
, 
position <class 'pynwb.core.VectorData'>
Fields:
  description: Animal spatial position during an epoch
, 
speed <class 'pynwb.core.VectorData'>
Fields:
  description: Animal speed during an epoch
, 
head_dir <class 'pynwb.core.VectorData'>
Fields:
  description: Animal head direction during an epoch
, 
task <class 'pynwb.core.VectorData'>
Fields:
  description: Behavioral task for this epoch
, 
events <class 'pynwb.core.VectorData'>
Fields:
  description: Possibly-overlapping behavioral events in this epoch
, 
graph_path <class 'pynwb.core.VectorData'>
Fields:
  description: Non-overlapping graph path for this epoch
)
  description: 
  id: None



Position in epoch 2
------------

Position